In [1]:
import os
os.environ["CUDA_DEVICE_OREDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
#from torchsummary import summary
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random
print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

from copy import deepcopy

1.8.0 0.2.2
TITAN RTX
11.1


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


In [3]:
# randomness 제어 
# https://hoya012.github.io/blog/reproducible_pytorch/
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

In [4]:
# dataset load

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
dataset_path = "/media/2/Network/Imagenet_dup/"
retrain_model_path = "/media/2/hwbae0326/F4F/models/"
# imagenet data load
train_dataset = dataset.ImageFolder(root=dataset_path+"train",
                                       transform=transform)
val_dataset = dataset.ImageFolder(root=dataset_path+"val",
                                       transform=transform)
'''
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=64,
                                        shuffle=False,
                                        num_workers=4)
'''
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=64,
                                        shuffle=True,
                                        num_workers=4) # for using subset
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                        batch_size=64,
                                        shuffle=True,
                                        num_workers=4)
print(len(train_dataloader),len(val_dataloader))

20019 782


In [5]:
# model 로드 
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)

#if torch.cuda.device_count() > 1:
#    vgg16_bn = nn.DataParallel(vgg16_bn)

In [6]:
# only conv5_1 is trainable
''' hard coding way to freeze weigth
ct = 0
layer_level = 1
back_layer = []
for layer in vgg16_bn.children():
#    print(ct,layer)
    if layer_level is 1:
        for layer_1 in layer:    
            print("layer 1_",ct,layer_1)
            for i,param in enumerate(layer_1.parameters()):
                if ct != 34:
                    param.requires_grad = False
                    #print(ct,"layer false")
                else :
                    if i == 0:
                        param.requires_grad = True
                        print("=====",param.shape)
                        print("%d-%d_layer true"%(ct,i))
                    else :
                        param.requires_grad = False
                        print("=====",param.shape)
                        print("%d-%d_layer false"%(ct,i))
                        
            ct +=1
    elif layer_level is 2:
    #    print("layer 2",layer)  
        for param in layer_1.parameters():
            param.requires_grad = False
            #print(ct,"layer false")
        ct +=1
    elif layer_level is 3:
        for layer_1 in layer: 
    #        print("layer 3",layer_1) 
            for param in layer_1.parameters():
                param.requires_grad = False
                #print(ct,"layer false")
            ct+=1
    layer_level +=1
'''
    

' hard coding way to freeze weigth\nct = 0\nlayer_level = 1\nback_layer = []\nfor layer in vgg16_bn.children():\n#    print(ct,layer)\n    if layer_level is 1:\n        for layer_1 in layer:    \n            print("layer 1_",ct,layer_1)\n            for i,param in enumerate(layer_1.parameters()):\n                if ct != 34:\n                    param.requires_grad = False\n                    #print(ct,"layer false")\n                else :\n                    if i == 0:\n                        param.requires_grad = True\n                        print("=====",param.shape)\n                        print("%d-%d_layer true"%(ct,i))\n                    else :\n                        param.requires_grad = False\n                        print("=====",param.shape)\n                        print("%d-%d_layer false"%(ct,i))\n                        \n            ct +=1\n    elif layer_level is 2:\n    #    print("layer 2",layer)  \n        for param in layer_1.parameters():\n            p

In [7]:
tmp=np.array([1,45,6,32,])
tmp.argmax()
tmp = np.append(tmp,77)
model =None
model =vgg16_bn
tmp,type(model)
torch.save(model.state_dict(), retrain_model_path+"vgg16_bn_epoch_%d_state_dict.pt"%(tmp[1]))

In [8]:

feature_output=[] # extract 10 features from conv5_1 input
def error_injection(name,num_error,start_index):
    def hook(model,input):
        start = start_index
        end = start_index + num_error
        input[0][:, start:end]=0
        #print("shape :",input[0][0].size())
        if len(feature_output) < 10:
            #print("error_injection, input_shape:",input[0][0][start:end].size(),input[0][:, start:end].size())
            feature_output.append(input[0][0].cpu())
    return hook
num_error = 128
error_index = 0
def hook_register(vgg16_bn,error_index,num_error):
    param_list = []
    for name,parameter in vgg16_bn.named_parameters():
        if "features.34.weight" in name:
        #print(name,"size:",parameter.size())
            param_list.append(parameter)
    for name,layer in vgg16_bn.named_modules():
    #print(name)
        if "features.34" in name  and isinstance(layer, torch.nn.modules.conv.Conv2d):
            print(name,layer) # target layer Conv5_1
            handle = layer.register_forward_pre_hook(error_injection(name,num_error,error_index))
    return param_list
param_list = hook_register(vgg16_bn,error_index,num_error)

features.34 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [9]:
# evaluation phasetraining
def eval(model,dataloader,epoch):

    model.cuda()
    model.eval()
    total = 0
    correct =0
    with torch.no_grad():
        print("======eval start=======")
        for i, data in enumerate(dataloader):
            inputs,labels = data
            inputs,labels = inputs.to(device), labels.to(device)
        
            y_hat = model(inputs)
            _, predicted = torch.max(y_hat.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            if(i%200 == 199):
                print("step : {} / {}".format(i + 1, int(len(val_dataset)/labels.size(0))))
                #print(".",end="")
        print("")
    acc = 100*correct/total
    print("%dth epoch acc of %s on imagenet : %.4f %%" %(epoch, model.__class__.__name__,acc)) 
    print("======eval  end ======")  
    return acc
#torch.save(vgg16_bn.state_dict(), retrain_model_path+"test_vgg16_bn_state_dict.pt")
def model_copy(model):
    return deepcopy(model.state_dict())

In [13]:
# training
# info
def training(model,train_dataloader,optimizer,loss_fn,error_idx,num_error,max_epochs=30,subset=False):
    print("before training")
    tmp_acc = eval(model,val_dataloader,0)
    model.to(device)
    info = [0.0,0.0] # high_acc, releated_epoch
    top_acc_model= None
    tolerance = 4
    for epoch in range(max_epochs):
        running_loss = 0.0
        print("=====epoch %d start======"%(epoch))
        model.train()
        cumulation= 0
        for i, data in enumerate(train_dataloader):
            inputs,labels = data
            inputs,labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
        
            y_pred = model(inputs)
            # compute loss 
            loss = loss_fn(y_pred,labels)
            #print(epoch,i, loss.item())
            
            if not torch.isfinite(loss):
                print("WARNING: non-finite loss, ending training")
                exit(1)
            else :
                running_loss += loss.item()
            loss.backward()
            optimizer.step()
            lr_sche.step()
            if i % 100 == 99: 
                print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/100)) 
                running_loss = 0.0
            if subset is True and i>=400:
                break
            
        tmp_acc = eval(model,val_dataloader,epoch)
        if tmp_acc > info[0]:
            info[0] = tmp_acc
            info[1] = epoch
            top_acc_model = model_copy(model)
        elif cumulation < tolerance : # validation fail
            cumulation +=1
        else:
            cumulation = 0
            optimizer.param_groups[0]['lr'] *= 0.5
            print("learning rate decreased to %f"%(optimizer.param_groups[0]['lr']))
        print("=====epoch %d end======"%(epoch))
    # save high_acc model
    torch.save(top_acc_model, 
               retrain_model_path+"%d~%d_pkt_err_vgg16_bn_acc_%.3f_epoch_%d_state_dict.pt"
               %(error_idx,error_idx+num_error,info[0],info[1]))           

In [14]:
loss_fn = torch.nn.CrossEntropyLoss(size_average = None).to(device)
optimizer = torch.optim.SGD(param_list,lr=0.01,weight_decay=1e-4)
lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.99)
max_epochs = 30

In [ ]:
num_error = 128
for error_idx in range(0,512,num_error):
    vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)
    param_list = hook_register(vgg16_bn,error_idx,num_error)
    print("**********error_idx :%d, num error :%d**********"%(error_idx, num_error))
    training(vgg16_bn,train_dataloader,optimizer,loss_fn,error_idx,num_error, max_epochs,True) 
    

features.34 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
**********error_idx :0, num error :128**********
before training
======eval start=======
step : 100 / 781
step : 200 / 781
step : 300 / 781
step : 400 / 781
step : 500 / 781
step : 600 / 781
step : 700 / 781

0th epoch acc of VGG on imagenet : 63.2920 %
======eval  end ======
=====epoch 0 start======
[1,   100] loss: 0.901
[1,   200] loss: 0.933
[1,   300] loss: 0.887
[1,   400] loss: 0.902
======eval start=======
step : 100 / 781
step : 200 / 781
step : 300 / 781
step : 400 / 781
step : 500 / 781
step : 600 / 781
step : 700 / 781

0th epoch acc of VGG on imagenet : 69.9520 %
======eval  end ======
=====epoch 0 end======
=====epoch 1 start======
[2,   100] loss: 0.920
[2,   200] loss: 0.906
[2,   300] loss: 0.894
[2,   400] loss: 0.905
======eval start=======
step : 100 / 781
step : 200 / 781
step : 300 / 781
step : 400 / 781
step : 500 / 781
step : 600 / 781
step : 700 / 781

1th epoch acc of VGG on imagen

In [ ]:
print("baseline evaluation")
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True) # not error inserted
# evaluation phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)

vgg16_bn.cuda()
vgg16_bn.eval()
total = 0
correct =0
with torch.no_grad():
    print("eval start")
    for i, data in enumerate(val_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16_bn(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if(i%50 == 49):
            print("step : {} / {}".format(i + 1, int(len(val_dataset)/labels.size(0))))
            #print(".",end="")
    print("")
print("acc of vgg16 not shuffle on imagenet : %.4f %%" %(100*correct/total))    